In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import unicodedata
import re
import time

In [2]:
# Import dataset
ds = pd.read_csv('train.csv')
ds = ds[ds['language']=='English']

In [3]:
# Preprocess data by removing non-alphanumeric characters and tokenizing 
def preprocessSentence(s):
    s = s.lower().strip()
    s = re.sub(r"([?.!,¿])", r" \1 ", s)
    s = re.sub(r'[" "]+', " ", s)
    s = re.sub(r"[^a-zA-Z?.!,¿]+", " ", s)
    s = s.strip()
    s = '<start> ' + s + ' <end>'
    return s

def createTokenizer(data):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(data)
    return tokenizer

def tokenizeData(data,tokenizer):
    tensor = tokenizer.texts_to_sequences(data)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
    return tensor

premises = []
for premise in ds['premise']:
    premises.append(preprocessSentence(premise))

hypotheses = []
for hypothesis in ds['hypothesis']:
    hypotheses.append(preprocessSentence(hypothesis))
    
tokenizer = createTokenizer(premises+hypotheses)
premise_tensor = tokenizeData(premises,tokenizer)
hypothesis_tensor = tokenizeData(hypotheses,tokenizer)
ph_tensor = tf.concat([premise_tensor,hypothesis_tensor],1)

In [4]:
# Create tf data pipeline
BUFFER_SIZE = len(ph_tensor)
BATCH_SIZE = 64
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 100
units = 100
vocab_size = len(tokenizer.word_index)+1
split_size = premise_tensor.shape[1]

train_input, val_input, train_targ, val_targ = train_test_split(ph_tensor.numpy(),ds['label'].values,test_size=0.3)

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_input, train_targ)) #.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

val_dataset = tf.data.Dataset.from_tensor_slices((val_input,val_targ)) #.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [9]:
# Define Encoder + Decoder architectures


class Encoder(tf.keras.Model):
    
    def __init__(self,vocab_size,embedding_dim,enc_units,batch_size):
        super(Encoder,self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
        
    def call(self,x,hidden):
        x = self.embedding(x)
        output, state = self.gru(x,initial_state=hidden)
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size,self.enc_units))


class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self,units):
        super(BahdanauAttention,self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
    
    def call(self,query,values):
        query_with_time_axis = tf.expand_dims(query,1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score,axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights
    
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(3)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
                
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights
    
    
class RTEAttention(tf.keras.Model):
    def __init__(self,vocab_size,embedding_dim,enc_units,dec_units,batch_size,split_size, **kwargs):
        super(RTEAttention, self).__init__(**kwargs)
        
        #self.original_dim = original_dim
        self.split_size = split_size
        self.encoder = Encoder(vocab_size,embedding_dim,enc_units,batch_size)
        self.decoder = Decoder(vocab_size, embedding_dim, dec_units, batch_size)
        
    def call(self,inp):
        enc_hidden = self.encoder.initialize_hidden_state()
        premise, hypothesis = tf.split(inp,[self.split_size,-1],axis=1)
        enc_output, enc_hidden = self.encoder(premise,enc_hidden)
        dec_hidden = enc_hidden
        
        for t in range(hypothesis.shape[1]):
            dec_input = tf.expand_dims(hypothesis[:,t],1)
            predictions, dec_hidden, _ = self.decoder(dec_input, dec_hidden, enc_output)
            
        return predictions

In [10]:
model = RTEAttention(vocab_size,embedding_dim,units,units,BATCH_SIZE,split_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model.fit(train_dataset, epochs=5,validation_data=val_dataset,verbose=1)

Epoch 1/5
75/75 [==============================] - 189s 3s/step - loss: 1.0992 - accuracy: 0.3438 - val_loss: 1.0972 - val_accuracy: 0.3662
Epoch 2/5
75/75 [==============================] - 175s 2s/step - loss: 1.0985 - accuracy: 0.3440 - val_loss: 1.0974 - val_accuracy: 0.3662
Epoch 3/5
75/75 [==============================] - 176s 2s/step - loss: 1.0985 - accuracy: 0.3467 - val_loss: 1.0974 - val_accuracy: 0.3662
Epoch 4/5
75/75 [==============================] - 176s 2s/step - loss: 1.0971 - accuracy: 0.3679 - val_loss: 1.1058 - val_accuracy: 0.2646
Epoch 5/5
75/75 [==============================] - 175s 2s/step - loss: 1.0482 - accuracy: 0.4508 - val_loss: 1.1898 - val_accuracy: 0.2212


In [ ]:
encoder = Encoder(vocab_size, embedding_dim, units, BATCH_SIZE)

# sample input
example_input_batch, example_target_batch = next(iter(train_dataset))

sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch[:,:split_size], sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

decoder = Decoder(vocab_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, sample_decoder_state, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))
print ('Decoder state shape: (batch_size, vocab size) {}'.format(sample_decoder_state.shape))


optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#@tf.function
# def train_step(inp, targ, enc_hidden, split_size):
    
#     premise, hypothesis = tf.split(inp,[split_size,-1],axis=1)

#     with tf.GradientTape() as tape:
#         enc_output, enc_hidden = encoder(premise,enc_hidden)
        
#         dec_hidden = enc_hidden
        
        
#         for t in range(hypothesis.shape[1]):
#             dec_input = tf.expand_dims(hypothesis[:,t],1)
#             predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                    
#         loss = loss_object(targ,predictions)
        
#         variables = encoder.trainable_variables + decoder.trainable_variables

#         gradients = tape.gradient(loss, variables)

#         optimizer.apply_gradients(zip(gradients, variables))

#         return loss


EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    batch_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        
        batch_loss = train_step(inp, targ, enc_hidden, tf.constant(split_size))
        total_loss += batch_loss

        if batch % 1 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,batch,batch_loss))
            
    # saving (checkpoint) the model every 2 epochs
#     if (epoch + 1) % 2 == 0:
#         checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))